In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 200 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 21
piter = 10
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [4]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}

In [5]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [6]:
#Read-in deterministic data observations for use in inference.
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', 4, t, 0.1)
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', 5, t, 0.1)
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', 5, t, 0.1)

In [7]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [8]:
obs_model_CON_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_CON[:-1, :], SCALE = obs_error_CON[:, :-1])
obs_model_AWB_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB[:-1, :], SCALE = obs_error_AWB[:, :-1])
obs_model_AWB_ECA_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB_ECA[:-1, :], SCALE = obs_error_AWB_ECA[:, :-1])

In [9]:
obs_model_CON_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_CON, SCALE = obs_error_CON, GET_CO2 = get_CO2_CON)
obs_model_AWB_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB, SCALE = obs_error_AWB, GET_CO2 = get_CO2_AWB)
obs_model_AWB_ECA_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB_ECA, SCALE = obs_error_AWB_ECA, GET_CO2 = get_CO2_AWB)

In [10]:
net = SDEFlow(devi, batch_size, obs_model_CON_no_CO2, state_dim_SCON, t, dt, n).to(devi)
C_PATH, log_prob = net()
print('C_PATH:', C_PATH)
print('C_PATH.size():', C_PATH.size())

C_PATH: tensor([[[0.6603, 0.3864, 0.6279],
         [0.8639, 1.3072, 0.4864],
         [0.6541, 0.5509, 0.6598],
         ...,
         [0.4915, 0.9184, 0.6650],
         [0.7113, 0.6336, 0.4755],
         [0.5844, 0.4962, 0.2209]],

        [[0.6603, 0.6821, 0.6889],
         [1.0176, 0.2618, 0.5733],
         [2.3672, 0.4712, 0.5619],
         ...,
         [1.4129, 0.6689, 0.7332],
         [0.3559, 0.7658, 0.4844],
         [0.9403, 0.7034, 0.1836]]], grad_fn=<AddBackward0>)
C_PATH.size(): torch.Size([2, 1000, 3])


In [11]:
test_CO2 = get_CO2_CON(C_PATH, t_span_tensor[:, 1:, :], SCON_C_params_dict, temp_gen, temp_ref)
print(test_CO2)
torch.cat([C_PATH, test_CO2], -1)

tensor([[[0.0014],
         [0.0048],
         [0.0022],
         ...,
         [0.0068],
         [0.0047],
         [0.0035]],

        [[0.0025],
         [0.0011],
         [0.0019],
         ...,
         [0.0051],
         [0.0056],
         [0.0050]]], grad_fn=<LowerBoundBackward>)


tensor([[[6.6030e-01, 3.8635e-01, 6.2792e-01, 1.4455e-03],
         [8.6388e-01, 1.3072e+00, 4.8643e-01, 4.8231e-03],
         [6.5408e-01, 5.5087e-01, 6.5978e-01, 2.1872e-03],
         ...,
         [4.9153e-01, 9.1841e-01, 6.6495e-01, 6.8269e-03],
         [7.1134e-01, 6.3357e-01, 4.7546e-01, 4.6603e-03],
         [5.8437e-01, 4.9625e-01, 2.2090e-01, 3.5462e-03]],

        [[6.6030e-01, 6.8210e-01, 6.8895e-01, 2.4845e-03],
         [1.0176e+00, 2.6179e-01, 5.7332e-01, 1.0508e-03],
         [2.3672e+00, 4.7118e-01, 5.6189e-01, 1.9065e-03],
         ...,
         [1.4129e+00, 6.6894e-01, 7.3316e-01, 5.0997e-03],
         [3.5593e-01, 7.6585e-01, 4.8435e-01, 5.5817e-03],
         [9.4029e-01, 7.0341e-01, 1.8356e-01, 4.9962e-03]]],
       grad_fn=<CatBackward>)

In [14]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1])
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx mean =', x_with_CO2.mean(-2))
                    print('\nx =', x_with_CO2)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx mean =', x_with_CO2.mean(-2))
                    print('\nx =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [15]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000009169.86875. Best norm loss value is: 91698.6875.

x mean = tensor([[0.6565, 0.7434, 0.8904, 0.0032],
        [0.6605, 0.7463, 0.8899, 0.0032]], grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0111e-01, 7.2878e-01, 7.9769e-01, 2.6630e-03],
         [2.5569e-01, 9.3656e-01, 8.7754e-01, 3.5254e-03],
         ...,
         [7.0084e-01, 7.5433e-01, 9.6218e-01, 5.7426e-03],
         [7.0144e-01, 7.3601e-01, 1.3723e+00, 5.6397e-03],
         [9.9565e-01, 9.2130e-01, 9.9927e-01, 6.7433e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0111e-01, 6.0170e-01, 6.8307e-01, 2.2042e-03],
         [3.9953e-01, 8.9332e-01, 7.6644e-01, 3.3553e-03],
         ...,
         [6.9988e-01, 6.8121e-01, 8.7385e-01, 5.1910e-03],
         [7.0439e-01, 8.1732e-01, 8.7033e-01, 6.0737e-03],
         [8.8126e-01, 7.2544e-01, 9.9640e-01, 5.3730e-03]]],
       grad_fn=<CatBackward>)



Train Diffusion:  48%|████▊     | 10/21 [00:08<00:08,  1.24it/s]

Moving average norm loss at 10 iterations is: 87991.8109375. Best norm loss value is: 84303.625.

x mean = tensor([[3.9621e+00, 3.6008e-01, 5.9928e-01, 1.6631e-03],
        [3.8803e+00, 3.5172e-01, 5.9937e-01, 1.6312e-03]],
       grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.9839e-01, 7.2605e+00, 4.2156e-01, 2.5370e-02],
         [3.8044e-01, 9.4663e-01, 4.3321e-01, 3.4999e-03],
         ...,
         [1.3946e-01, 3.8716e-01, 8.6575e-01, 3.0429e-03],
         [1.0618e+01, 4.1835e-01, 7.4744e-01, 3.7229e-03],
         [1.0938e+00, 4.2158e-01, 5.7957e-01, 3.1518e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.9839e-01, 6.1658e-01, 3.8812e-01, 2.2149e-03],
         [7.5818e-01, 2.5258e-01, 3.9245e-01, 9.8634e-04],
         ...,
         [4.2353e+00, 4.1546e-01, 7.4602e-01, 3.4281e-03],
         [7.0545e-01, 4.2460e-01, 8.2073e-01, 3.2773e-03],
         [7.4987e+00, 3.8737e-01, 7.3680e-01, 3.2789e-03]]],


Train Diffusion:  95%|█████████▌| 20/21 [00:15<00:00,  1.36it/s]

Moving average ELBO loss at 20 iterations is: 388770.928125. Best ELBO loss value is: 230084.296875.

x mean = tensor([[2.3984, 0.9379, 1.0552, 0.0041],
        [2.3785, 0.9646, 1.1159, 0.0041]], grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.8212e-01, 5.4075e-01, 1.5652e+00, 2.1162e-03],
         [6.5315e-01, 5.1515e-01, 1.2469e+00, 2.0596e-03],
         ...,
         [2.5964e+00, 2.0437e-01, 2.7275e+00, 2.3929e-03],
         [4.6825e+00, 1.2943e+00, 2.5549e+00, 1.0136e-02],
         [3.4685e+00, 1.7497e+00, 1.4915e+00, 1.2772e-02]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [9.2089e-01, 6.4346e+00, 9.2817e-02, 2.2450e-02],
         [1.3013e+00, 3.6343e+00, 2.0149e-01, 1.3222e-02],
         ...,
         [9.5754e-01, 1.1451e+00, 1.8395e-01, 8.3446e-03],
         [1.0469e+00, 3.4308e-01, 2.1871e-01, 2.5467e-03],
         [6.4507e-01, 3.3090e-01, 2.0543e-01, 2.3937e-03]]],
       grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 21/21 [00:16<00:00,  1.28it/s]
